In [1]:
import os
from lib.train_AI_lib import *
import cv2
import numpy as np

In [2]:
os.chdir('..')
print(os.getcwd())

c:\Users\kevin\Documents\GitHub\APS360-2020Summer-Project


In [3]:
def createMask(bboxes, imgRes=(1024, 2014)):
    globalMask = np.zeros(imgRes, dtype=bool)
    for bbox in bboxes:
        oneMask = np.zeros(imgRes, dtype=bool)
        bbox = [int(number) for number in bbox]
        # Don't ask me why this flip works, numpy and cv2 take images in differently and it's a pain to try and figure out the transformations    
        oneMask[bbox[0]:(bbox[0]+bbox[2]), imgRes[1]-(bbox[1]+bbox[3]):imgRes[1]-bbox[1]] = np.ones((bbox[2], bbox[3]), dtype=bool)
        globalMask = np.bitwise_or(globalMask, oneMask)
    globalMask = np.rot90(globalMask, 1)
    return(globalMask)

def genBBoxImages(dictPath='saved/splitData', inPath='data/working-wheat-data/train', oPath='saved/bboxMasked'):
    try: os.makedirs(oPath)  # Make the requested oPath
    except FileExistsError: None
    except: print("error creating folder {}".format(oPath)); return(0)
    else: None
    
    trainData = torch.load(dictPath+"/trainData")
    valData   = torch.load(dictPath+"/valData"  )
    testData  = torch.load(dictPath+"/testData" )
    trainData.update(valData); trainData.update(testData)
    imgPath   = inPath+'/'
    keyList   = list(trainData.keys())

    for i, imgName in enumerate(keyList):
        img  = cv2.imread(imgPath+imgName)
        bbox = trainData[imgName]
        mask = createMask(bbox, np.shape(img)[:-1])
        img[:, :, 0] = np.multiply(img[:, :, 0], mask)
        img[:, :, 1] = np.multiply(img[:, :, 1], mask)
        img[:, :, 2] = np.multiply(img[:, :, 2], mask)

        if i%100==0: print("Converted {:.2f}%".format(100*i/len(keyList)))
        cv2.imwrite(oPath+'/'+imgName, img)

In [4]:
#genBBoxImages()

In [5]:
class autEncA(nn.Module):
    def __init__(self, name):
        super(autEncA, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 5, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(5, 10, 3, stride=2, padding=1),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(10, 5, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(5, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
calcNoParam(autEncA(''))

1193


In [6]:
class autEncB(nn.Module):
    def __init__(self, name):
        super(autEncB, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 5, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(5, 10, 4, stride=2, padding=1),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(10, 5, 4, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(5, 3, 4, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
calcNoParam(autEncB(''))

2103


In [8]:
batchsize=64; epoch=30; lr=0.001
net = autEncA('autEncA'); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize, tempPath='temp/auto', mode='auto', altArg={'compPath':'saved/bboxMasked'})
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='AutoEnc')

Converting Training Images
Converted 0.00%
Converted 5.48%
Converted 10.96%
Converted 16.44%
Converted 21.91%
Converted 27.39%
Converted 32.87%
Converted 38.35%
Converted 43.83%
Converted 49.31%
Converted 54.78%
Converted 60.26%
Converted 65.74%
Converted 71.22%
Converted 76.70%
Converted 82.18%
Converted 87.66%
Converted 93.13%
Converted 98.61%
Converting Validation Images
Converted 0.00%
Converted 43.86%
Converted 87.72%
Converting Testing Images
Converted 0.00%
Converted 43.86%
Converted 87.72%
Converting Complete


RuntimeError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 6.00 GiB total capacity; 3.84 GiB already allocated; 420.63 MiB free; 3.85 GiB reserved in total by PyTorch)